In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm

In [2]:
def get_model(nodes_in_hidden_layers, dropout, nr_of_inputs):
    model = Sequential()
    for i in range(len(nodes_in_hidden_layers)):
        if i == 0:
            model.add(Dense(nodes_in_hidden_layers[i], activation='swish', input_shape=(nr_of_inputs,)))  # Input and hidden layer
        else:
            if dropout != 0:
                model.add(Dropout(dropout))
            model.add(Dense(nodes_in_hidden_layers[i], activation='swish'))  # Additional hidden layers
    model.add(Dense(1, activation="linear"))  # Output layer
    model.compile(optimizer=Adam(learning_rate=0.001), loss="mean_squared_error", metrics=("mean_squared_error"))
    return model

In [3]:
def get_all_features(data):
    station = np.eye(19)[data["Base Station"]]
    year = np.eye(4)[data["Year"] - 2015]
    month = np.eye(12)[data["Month"] - 1]
    day = np.eye(31)[data["Day"] - 1]
    week = np.eye(53)[data["Week"] - 1]
    weekday = np.eye(7)[data["Weekday"] - 1]
    hour = np.eye(24)[data["Hour"]]
    daytime = np.zeros((len(data), 6))

    # Loop over the four-hour periods and sum the one hot encoded values of each hour in the period
    for i in range(6):
        daytime[:, i] = np.sum(hour[:, i*4:(i+1)*4], axis=1)

    x = np.concatenate([
        station,
        year, 
        month, 
        day, 
        week, 
        weekday,
        hour,
        daytime], axis=1)
    return x

In [4]:
def get_features(data):
    station = np.eye(19)[data["Base Station"]]
    year = np.eye(4)[data["Year"] - 2015]
    month = np.eye(12)[data["Month"] - 1]
    day = np.eye(31)[data["Day"] - 1]
    #week = np.eye(53)[data["Week"] - 1]
    weekday = np.eye(7)[data["Weekday"] - 1]
    hour = np.eye(24)[data["Hour"]]
    daytime = np.zeros((len(data), 6))

    # Loop over the four-hour periods and sum the one hot encoded values of each hour in the period
    for i in range(6):
        daytime[:, i] = np.sum(hour[:, i*4:(i+1)*4], axis=1)

    x = np.concatenate([
        station,
        year, 
        month, 
        day, 
        #week, 
        weekday,
        hour,
        daytime], axis=1)
    return x

In [5]:
def get_original_features(data):
    def get_daytime(hour, period_hours):
        periods = int(24/period_hours)
        for i in range(periods):
            start_hour = i
            end_hour = (i+1) * period_hours
            if start_hour <= hour < end_hour:
                return i
        return 6

    # Map the Hour column to the Daytime column using the map_hour_to_daytime function
    data["Daytime"] = data["Hour"].apply(lambda x: get_daytime(x, 4))

    return data[["Base Station", "Year", "Month", "Day", "Week", "Weekday", "Hour", "Daytime"]].values

In [6]:
def revert_all_features(data):
    base_station = np.argmax(data[:, :19], axis=1) 
    year = np.argmax(data[:, 19:23], axis=1) + 2015
    month = np.argmax(data[:, 23:35], axis=1) + 1
    day = np.argmax(data[:, 35:66], axis=1) + 1
    week = np.argmax(data[:, 66:119], axis=1) + 1
    weekday = np.argmax(data[:, 119:126], axis=1) + 1
    hour = np.argmax(data[:, 126:150], axis=1)
    daytime = np.argmax(data[:, 150:], axis=1)


    return list(zip(base_station, year, month, day, week, weekday, hour, daytime))

In [7]:
def revert_features(data):
    base_station = np.argmax(data[:, :19], axis=1) 
    year = np.argmax(data[:, 19:23], axis=1) + 2015
    month = np.argmax(data[:, 23:35], axis=1) + 1
    day = np.argmax(data[:, 35:66], axis=1) + 1
    #week = np.argmax(data[:, 66:119], axis=1) + 1
    weekday = np.argmax(data[:, 66:73], axis=1) + 1
    hour = np.argmax(data[:, 73:97], axis=1)
    daytime = np.argmax(data[:, 97:], axis=1)

    return list(zip(base_station, year, month, day, weekday, hour, daytime))

In [15]:
from datetime import datetime

def train_test_split_date(data):
    start_date = datetime(2017, 8, 14)
    end_date = datetime(2017, 8, 21)

    mask = (pd.to_datetime(data['Year'].astype(str) + '-' + data['Month'].astype(str) + '-' + data['Day'].astype(str))\
    .dt.floor('d')\
    .between(start_date, end_date, inclusive=True))

    data_train = data[~mask]    
    data_test = data[mask]

    return data_train, data_test

In [9]:
def preprocess(data, s):
    if s != None:
        data_station = data[data["Base Station"] == s]
    x = get_all_features(data_station)
    y = data_station["Incidents"].to_numpy()
    return x, y

In [10]:
DISTRIBUTION_FILE = "data/incident_distribution/station.csv"

data = pd.read_csv(DISTRIBUTION_FILE, encoding='utf-8', escapechar='\\', parse_dates=True)

In [16]:
data_train, data_test = train_test_split_date(data)

C:\Users\erikm\AppData\Local\Temp\ipykernel_19520\773479429.py:7: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  mask = (pd.to_datetime(data['Year'].astype(str) + '-' + data['Month'].astype(str) + '-' + data['Day'].astype(str))\


In [ ]:
X_train, Y_train = preprocess(data_train, None)
X_test_s, Y_test_s = preprocess(data_test, None)

layers = [128, 32, 8]
model = get_model(layers, 0.4, len(X_train[0]))
es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=1, patience=1)
model.fit(X_train, Y_train, epochs=10, verbose=1, callbacks=[es])

In [ ]:
def cross_validation(x_train, y_train, nodes, epochs, dropout, patience):
    kf = KFold(n_splits=5)
    ce = []
    for train_index, validation_index in kf.split(x_train):
        model = get_model(nodes, dropout, len(x_train[0]))
        es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=1, patience=patience)
        history = model.fit(x_train[train_index], y_train[train_index], validation_data=(x_train[validation_index], y_train[validation_index]), epochs=epochs, verbose=1, callbacks=[es])
        ce.append(min(history.history['mean_squared_error']))
    avg_ce = sum(ce) / len(ce)
    return avg_ce

X_train, Y_train = preprocess(data, config)
X_test, Y_test = preprocess(data_test, config)

model = get_model(config["layers"], config["dropout"], len(X_train[0]), config["lr"])
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=config["patience"])

x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=config["seed"])

In [17]:
predictions = []
Y_test = []
X_test = []

for s in range(0, 19):
    X_train, Y_train = preprocess(data_train, s)
    X_test_s, Y_test_s = preprocess(data_test, s)

    layers = [128, 64, 16]
    model = get_model(layers, 0.4, len(X_train[0]))
    es = EarlyStopping(monitor='val_mean_squared_error', mode='min', verbose=1, patience=1)
    model.fit(X_train, Y_train, epochs=20, verbose=1, validation_split=0.2, shuffle=False, callbacks=[es])

    """# Fit the Poisson regression model
    model = sm.GLM(Y_train, X_train, family=sm.families.Poisson())
    model = sm.GLM(Y_train, X_train, family=sm.families.NegativeBinomial())
    model = sm.ZeroInflatedGeneralizedPoisson(Y_train, X_train, exog_infl=X_train, inflation='logit')
    model = model.fit() """

    predictions_station = model.predict(X_test_s, verbose=0)
    print(mean_squared_error(Y_test_s, predictions_station))

    predictions.extend(predictions_station)
    X_test.extend(X_test_s)
    Y_test.extend(Y_test_s)

predictions = np.array(predictions)
Y_test = np.array(Y_test)
X_test = np.array(X_test)

Epoch 1/20
872/872 [==============================] - 3s 2ms/step - loss: 0.3286 - mean_squared_error: 0.3286 - val_loss: 0.3347 - val_mean_squared_error: 0.3347
Epoch 2/20
872/872 [==============================] - 2s 2ms/step - loss: 0.3237 - mean_squared_error: 0.3237 - val_loss: 0.3352 - val_mean_squared_error: 0.3352
Epoch 2: early stopping
0.2786518059981515
Epoch 1/20
872/872 [==============================] - 2s 2ms/step - loss: 0.1885 - mean_squared_error: 0.1885 - val_loss: 0.1894 - val_mean_squared_error: 0.1894
Epoch 2/20
872/872 [==============================] - 2s 2ms/step - loss: 0.1852 - mean_squared_error: 0.1852 - val_loss: 0.1904 - val_mean_squared_error: 0.1904
Epoch 2: early stopping
0.2167966753299507
Epoch 1/20
872/872 [==============================] - 2s 2ms/step - loss: 0.3603 - mean_squared_error: 0.3603 - val_loss: 0.4030 - val_mean_squared_error: 0.4030
Epoch 2/20
872/872 [==============================] - 2s 2ms/step - loss: 0.3561 - mean_squared_error: 0

In [ ]:
# Fit the Poisson regression model
model = sm.GLM(Y_train, X_train, family=sm.families.Poisson())
model = model.fit()

# Print the model summary
print(model.summary())

In [ ]:
predictions = model.predict(X_test)

In [18]:
mse = mean_squared_error(Y_test, predictions)
print("Mean squared error: ", mse)

X_test_reverted = revert_all_features(X_test)

print("Base Station", "Year", "Month", "Day", "Week", "Weekday", "Hour", "Daytime")
for i in range(len(predictions)):
    print(predictions[i], np.round(predictions[i], 2), Y_test[i], X_test_reverted[i])

Mean squared error:  0.5870708108655662
Base Station Year Month Day Week Weekday Hour Daytime
[0.2788195] [0.28] 0 (0, 2017, 8, 14, 33, 1, 0, 0)
[0.25585976] [0.26] 0 (0, 2017, 8, 14, 33, 1, 1, 0)
[0.23231603] [0.23] 0 (0, 2017, 8, 14, 33, 1, 2, 0)
[0.18940485] [0.19] 0 (0, 2017, 8, 14, 33, 1, 3, 0)
[0.17014697] [0.17] 0 (0, 2017, 8, 14, 33, 1, 4, 1)
[0.20112438] [0.2] 0 (0, 2017, 8, 14, 33, 1, 5, 1)
[0.22523484] [0.23] 0 (0, 2017, 8, 14, 33, 1, 6, 1)
[0.2433567] [0.24] 0 (0, 2017, 8, 14, 33, 1, 7, 1)
[0.4545184] [0.45] 1 (0, 2017, 8, 14, 33, 1, 8, 2)
[0.49258012] [0.49] 0 (0, 2017, 8, 14, 33, 1, 9, 2)
[0.53462636] [0.53] 0 (0, 2017, 8, 14, 33, 1, 10, 2)
[0.5066361] [0.51] 0 (0, 2017, 8, 14, 33, 1, 11, 2)
[0.4585451] [0.46] 0 (0, 2017, 8, 14, 33, 1, 12, 3)
[0.54555726] [0.55] 0 (0, 2017, 8, 14, 33, 1, 13, 3)
[0.49777353] [0.5] 1 (0, 2017, 8, 14, 33, 1, 14, 3)
[0.45236754] [0.45] 0 (0, 2017, 8, 14, 33, 1, 15, 3)
[0.4353681] [0.44] 0 (0, 2017, 8, 14, 33, 1, 16, 4)
[0.4005612] [0.4] 0 (0,

In [20]:
PREDICTIONS_FILE = "data/incidents_distribution_station_predictions_2.json"

def save_predictions(predictions, X_test_reverted):
    d = {}
    for station in range(0, 19):
        d[station] = {}
        for day in range(14, 22):
            d[station][day] = {}
            for hour in range(0, 24):
                d[station][day][hour] = {}
    for prediction, x in zip(predictions, X_test_reverted):

        station = x[0]
        day = x[3]
        hour = x[6]
    
        d[station][day][hour] = round(float(prediction), 3)
    print("Saving predictions to file...")
    with open(PREDICTIONS_FILE, 'w') as f:
        json.dump(d, f, indent=2)


save_predictions(predictions, X_test_reverted)

Saving predictions to file...
